In [23]:
from dateutil.parser import parse
from datetime import datetime
import subprocess
import yaml
import os

In [24]:
# Read batch yaml
with open('/ocean/bmoorema/research/SOG/sog-runsets/SOG_test.yaml', 'r') as f:
    data = yaml.load(f)

In [25]:
# Paths
results_path = os.path.join(data['paths']['runs_directory'], data['batch_name'])
initial_path = data['paths']['initial']
ctd_dir = data['initialization']['ctd_directory']
nuts_dir = data['initialization']['nutrients_directory']

In [26]:
# Check for forcing fields
vary_forcings = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
if 'forcing1' in data:
    if 'vary' in data['forcing']:
        vary_forcings = data['forcing']['vary']

# Check for river chemistry fields
vary_river_chem = False
river_TA, river_pH = [0, 0], [0]
if 'freshwater_chemistry' in data:
    vary_river_chem = True
    river_TA = data['freshwater_chemistry']['river_TA']
    river_pH = data['freshwater_chemistry']['river_pH']

In [27]:
# Build root directories
subprocess.call(['mkdir', 'editfiles'])
subprocess.call(['mkdir', results_path])

0

In [28]:
# Initialize batchfile dict
batchfile_dict = {
    'max_concurrent_jobs': data['max_concurrent_jobs'],
    'SOG_executable': os.path.join(data['paths']['SOG_code'], 'SOG'),
    'base_infile': os.path.join(data['paths']['SOG_code'], 'infile.yaml'),
    'jobs': [],
}

In [29]:
def vary_forcing_parser(key, editfile_dict, forcing_inputs, run_year, run_tag):
    """
    """

    # Extra string for cloud fraction
    string = ''
    if key is 'river':
        string = '_flows'
    elif key is 'cloud':
        string = '_fraction'
        
    # Vary forcing True or False
    if forcing_inputs[0]:
        editfile_dict[key + string] = {
            'value': True,
        }
        run_tag = run_tag + '_{}'.format(key)

        # Fixed
        if forcing_inputs[1]:
            editfile_dict[key + string + '_fixed'] = {
                'value': True,
            }
            editfile_dict[key + string + '_value'] = {
                'value': forcing_inputs[2],
            }
            run_tag = run_tag + '_fix{:.0f}'.format(forcing_inputs[2])

        # Fraction
        if forcing_inputs[3] != 1:
            editfile_dict[key + string + '_fraction'] = {
                'value': forcing_inputs[3],
            }
            run_tag = run_tag + '_frac{:.0f}'.format(forcing_inputs[3]*100)

        # Add
        if forcing_inputs[4] != 0:
            editfile_dict[key + string + '_addition'] = {
                'value': forcing_inputs[4],
            }
            run_tag = run_tag + '_add{:.0f}'.format(forcing_inputs[4])

        # Shift
        if forcing_inputs[5] != 0 or forcing_inputs[6] != 0:
            run_tag = run_tag + '_shift'

            # By year
            if forcing_inputs[5] != 0:
                editfile_dict[key + string + '_shift'] = {
                    'value': forcing_inputs[5] - run_year,
                }
                run_tag = run_tag + '{:d}'.format(forcing_inputs[5])

            # By day
            if forcing_inputs[6] != 0:
                editfile_dict[key + string + '_shift'] = {
                    'value': editfile_dict[key + string + '_shift'] + forcing_inputs[6]/365,
                }
                run_tag = run_tag + '_{:.0f}'.format(forcing_inputs[6])

    return editfile_dict, run_tag

In [30]:
def populate_editfile_dict(
    editfile_dict, datetimes, run_path, initial_path, ctd_dir, nuts_dir, init_files, hoffmueller=True
):
    """
    """
    
    # Parse timestamps
    datetime_start, datetime_end = map(parse, datetimes)
    
    editfile_dict.update({
        'initial_conditions': {
            'init_datetime': {
                'value': datetime_start.strftime('%Y-%m-%d %H:%M:%S'),
            },
            'CTD_file': {
                'value': os.path.join(initial_path, ctd_dir, init_files[0]),
            },
            'nutrients_file': {
                'value': os.path.join(initial_path, nuts_dir, init_files[1]),
            },
        },
        'end_datetime': {
            'value': datetime_end.strftime('%Y-%m-%d %H:%M:%S'),
        },
        'timeseries_results': {
            'std_physics': {
                'value': os.path.join(run_path, 'timeseries/std_phys_SOG.out'),
            },
            'user_physics': {
                'value': os.path.join(run_path, 'timeseries/user_phys_SOG.out'),
            },
            'std_biology': {
                'value': os.path.join(run_path, 'timeseries/std_bio_SOG.out'),
            },
            'user_biology': {
                'value': os.path.join(run_path, 'timeseries/user_bio_SOG.out'),
            },
            'std_chemistry': {
                'value': os.path.join(run_path, 'timeseries/std_chem_SOG.out'),
            },
            'user_chemistry': {
                'value': os.path.join(run_path, 'timeseries/user_chem_SOG.out'),
            },
        },
        'profiles_results': {
            'profile_file_base': {
                'value': os.path.join(run_path, 'profiles/SOG'),
            },
            'user_profile_file_base': {
                'value': os.path.join(run_path, 'profiles/SOG-user'),
            },
            'halocline_file': {
                'value': os.path.join(run_path, 'profiles/halo-SOG.out'),
            },
            'hoffmueller_file': {
                'value': os.path.join(run_path, 'profiles/hoff-SOG.dat'),
            },
            'user_hoffmueller_file': {
                'value': os.path.join(run_path, 'profiles/hoff-SOG-user.dat'),
            },
            'hoffmueller_start_year': {
                'value': datetime_start.year,
            },
            'hoffmueller_start_day': {
                'value': (datetime_start - datetime(datetime_start.year, 1, 1)).days + 2,
            },
            'hoffmueller_end_year': {
                'value': datetime_end.year,
            },
            'hoffmueller_end_day': {
                'value': (datetime_end - datetime(datetime_end.year, 1, 1)).days,
            },
        },
    })
    
    if not hoffmueller:
        editfile_dict['profiles_results'].update({
            'hoffmueller_end_year': {
                'value': datetime_start.year,
            },
            'hoffmueller_end_day': {
                'value': (datetime_start - datetime(datetime_start.year, 1, 1)).days + 3,
            },
        })

    return editfile_dict

In [32]:
# Loop through datetimes
for datetimes, init_files in zip(
    data['initialization']['datetimes'],
    data['initialization']['files']
):

    # Parse run_year
    run_year = parse(datetimes[0]).year + 1

    # Build directories
    subprocess.call(['mkdir', os.path.join('editfiles', str(run_year))])
    subprocess.call(['mkdir', os.path.join(results_path, str(run_year))])

    # Loop through forcing scenarios
    for vary_forcing in vary_forcings:
        
        # Initialize editfile dict and run tag
        editfile_dict = {}
        run_tag = '{:d}'.format(run_year)
        
        # Parse forcing vary inputs
        forcing_inputs = {'wind': vary_forcing[:7], 'river': vary_forcing[7:14], 'cloud': vary_forcing[14:]}
        if any([forcing_inputs['wind'][0], forcing_inputs['river'][0], forcing_inputs['cloud'][0]]):
            editfile_dict['vary'] = {}
            for key in ['wind', 'river', 'cloud']:
                editfile_dict['vary'], run_tag = vary_forcing_parser(
                    key, editfile_dict['vary'], forcing_inputs[key], run_year, run_tag
                )

        # Loop through river chemistry scenarios
        for TA in river_TA:
            for pH in river_pH:

                chem_tag = run_tag
                
                # Assign river chem params to editfile dict and update run_tag
                # (if they exist)
                if vary_river_chem:
                    editfile_dict['physics'] = {
                        'fresh_water': {'river_CO2_chemistry': {
                            'river_TA_record': TA[0],
                            'river_total_alkalinity': TA[1],
                            'river_pH': pH,
                        }}
                    }
                    if TA[0]:
                        TA_val = 'var'
                    else:
                        TA_val = str(TA[1])
                    chem_tag = chem_tag + '_TA{}pH{:.0f}'.format(TA_val, pH*10)

                # Build paths and directories
                editfile_path = os.path.join('editfiles', str(run_year), 'editfile_' + chem_tag)
                run_path = os.path.join(results_path, str(run_year), chem_tag)
                subprocess.call(['mkdir', run_path])

                # Build editfile
                editfile_dict = populate_editfile_dict(
                    editfile_dict, datetimes, run_path, initial_path, ctd_dir, nuts_dir, init_files, hoffmueller=True,
                )

                # Append run list in batchfile
                batchfile_dict['jobs'].append({chem_tag: {'edit_files': [editfile_path]}})

                # Write editfile yaml
                with open(editfile_path, 'w') as f:
                    yaml.dump(editfile_dict, f, default_flow_style=False)

2001_TAvarpH74
2001_TAvarpH77
2001_TAvarpH80
2001_TA500pH74
2001_TA500pH77
2001_TA500pH80
2001_TA750pH74
2001_TA750pH77
2001_TA750pH80
2001_TA1000pH74
2001_TA1000pH77
2001_TA1000pH80
2002_TAvarpH74
2002_TAvarpH77
2002_TAvarpH80
2002_TA500pH74
2002_TA500pH77
2002_TA500pH80
2002_TA750pH74
2002_TA750pH77
2002_TA750pH80
2002_TA1000pH74
2002_TA1000pH77
2002_TA1000pH80
2003_TAvarpH74
2003_TAvarpH77
2003_TAvarpH80
2003_TA500pH74
2003_TA500pH77
2003_TA500pH80
2003_TA750pH74
2003_TA750pH77
2003_TA750pH80
2003_TA1000pH74
2003_TA1000pH77
2003_TA1000pH80
2004_TAvarpH74
2004_TAvarpH77
2004_TAvarpH80
2004_TA500pH74
2004_TA500pH77
2004_TA500pH80
2004_TA750pH74
2004_TA750pH77
2004_TA750pH80
2004_TA1000pH74
2004_TA1000pH77
2004_TA1000pH80
2005_TAvarpH74
2005_TAvarpH77
2005_TAvarpH80
2005_TA500pH74
2005_TA500pH77
2005_TA500pH80
2005_TA750pH74
2005_TA750pH77
2005_TA750pH80
2005_TA1000pH74
2005_TA1000pH77
2005_TA1000pH80
2006_TAvarpH74
2006_TAvarpH77
2006_TAvarpH80
2006_TA500pH74
2006_TA500pH77
2006_TA500

In [37]:
with open('run_{}_{}'.format(data['machine'], data['batch_name']), 'w') as f:
    yaml.dump(batchfile_dict, f, default_flow_style=False)